In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple ( 'Customer', 'name fidelity' )

In [2]:
park = Customer ('Park', 100)
park

Customer(name='Park', fidelity=100)

In [17]:
class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [24]:
class Order:
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None) :
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion # 01 24*l|

  def total(self):
    """total 속성이 없으면 전체 계산할 값을 계산"""
    if not hasattr(self, '_total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    """할인금액 차감"""
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion. discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [25]:
class Promotion (ABC):
  """할인 혜택 클래스들의 형태를 선언"""

  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 반환"""
    pass

In [26]:
class FidelityPromo(Promotion):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  """I20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  """810종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [27]:
joe = Customer ('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [28]:
Order(joe,cart,FidelityPromo())

<Order total: 42.00 due: 42.00>

In [29]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [30]:
banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]

In [31]:
Order(joe, banana_cart, BulkItemPromo() )

<Order total: 30.00 due: 28.50>

In [32]:
long_order = [LineItem(str(item_code), 1, 1.0)
for item_code in range(10)]

In [33]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [34]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [44]:
class Order: # Context
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion # 101 2H*|

  def total(self):
    if not hasattr(self, '_total'):
      self ._total = sum(item.total() for item in self.cart)
    return self ._total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self) # promotion + 2|
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

  def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order. total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item. total() * 0.1
  return discount

def large_order_promo(order):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order. total() * 0.07
  return 0

In [45]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [46]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [47]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [49]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [50]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [51]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [52]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [53]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [54]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'park', '_2', '_i3', 'LineItem', '__init__', 'total', '_i4', 'Order', '__repr__', '_i5', 'Promotion', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', 'joe', 'ann', '_i8', '_i9', '_i10', '_i11', '_i12', 'cart', '_i13', '_13', '_i14', '_i15', '_15', '_i16', '_16', '_i17', '_i18', '_i19', '_i20', '_i21', '_i22', '_22', '_i23', '_23', '_i24', '_i25', '_i26', '_i27', '_i28', '_28', '_i29', '_29', '_i30', 'banana_cart', '_i31', '_31', '_i32', 'long_order', '_i33', '_33', '_i34', '_34', '_i35', '_i36', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i37', '_i38', '_i39', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_45', '_i46', '_46', '_i47', '_47', '_i48', '_48', '_i49', '_49', '_i50', 

In [55]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [56]:
promos = [globals () [name] for name in globals ()
          if name.endswith('_promo' )
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]